In [115]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import gc
gc.collect()

2819

In [116]:
def loadData(mu0, sigma0, mu1, sigma1, alpha0, alpha1):
    '''
    初始化数据集
    这里通过服从高斯分布的随机函数来伪造数据集
    :param mu0: 高斯0的均值
    :param sigma0: 高斯0的方差
    :param mu1: 高斯1的均值
    :param sigma1: 高斯1的方差
    :param alpha0: 高斯0的系数
    :param alpha1: 高斯1的系数
    :return: 混合了两个高斯分布的数据
    '''
    #定义数据集长度为1000
    length = 1000

    #初始化第一个高斯分布，生成数据，数据长度为length * alpha系数，以此来
    #满足alpha的作用
    data0 = np.random.normal(mu0, sigma0, int(length * alpha0))
    #第二个高斯分布的数据
    data1 = np.random.normal(mu1, sigma1, int(length * alpha1))

    #初始化总数据集
    #两个高斯分布的数据混合后会放在该数据集中返回
    dataSet = []
    #将第一个数据集的内容添加进去
    dataSet.extend(data0)
    #添加第二个数据集的数据
    dataSet.extend(data1)
    #对总的数据集进行打乱（其实不打乱也没事，只不过打乱一下直观上让人感觉已经混合了
    # 读者可以将下面这句话屏蔽以后看看效果是否有差别）
    random.shuffle(dataSet)

    #返回伪造好的数据集
    return dataSet

alpha0 = 0.3; mu0 = -2; sigmod0 = 0.5
alpha1 = 0.7; mu1 = 0.5; sigmod1 = 1

    #初始化数据集
dataSetList = loadData(mu0, sigmod0, mu1, sigmod1, alpha0, alpha1)

Y = np.array(dataSetList)

In [117]:
class GMM:
    def __init__(self, k, maxIter):
        self.k = k  #k个分模型
        self.maxIter = maxIter
        self.Y = None
        self.num_samples = None
        
    def fit(self, Y):
        
        self.Y = Y
        
        self.num_samples = len(Y)
        
        #初始化参数 随机更好，若指定为相同数 更新时反而不好。初始值很敏感
        mu = np.random.random(self.k)
        sigma = np.random.random(self.k)
        alpha = np.zeros(self.k)
        alpha[:] = 1 / self.k   #alpha 之和要为1
        
        for _ in range(self.maxIter):
            mu_, sigma_, alpha_ = mu.copy(), sigma.copy(), alpha.copy()
            eta_jk = self.E_step(mu, sigma, alpha) #E-step
            mu, sigma, alpha = self.M_step(mu, sigma, alpha, eta_jk) #M-step
            
            if self.finish(mu_, sigma_, mu, sigma): #迭代结束条件
                break
        
        return mu, sigma, alpha
            
        
    def Gaussian(self, y, mu, sigma):
        return 1 / (np.sqrt(2*np.pi)*sigma) * np.exp(-np.square(y-mu) / (2*np.square(sigma)))
    
    def E_step(self, mu, sigma, alpha):
        eta_jk = np.zeros((self.num_samples, self.k))
        for j in range(self.num_samples):
            denominator = 0
            for k in range(self.k):
                numerator = alpha[k] * self.Gaussian(self.Y[j], mu[k], sigma[k])
                denominator += numerator
                eta_jk[j, k] = numerator
            
            eta_jk[j, :] = eta_jk[j, :] / denominator
            
        return eta_jk
    
    def M_step(self, mu, sigma, alpha, eta_jk):
        for k in range(self.k):
            sigma[k] = np.sqrt(eta_jk[:, k].dot(np.square((self.Y-mu[k])).T) / np.sum(eta_jk[:, k]))
            mu[k] = eta_jk[:, k].dot(self.Y.T) / np.sum(eta_jk[:, k])
            alpha[k] = np.sum(eta_jk[:, k]) / self.num_samples
        return mu, sigma, alpha
    
    def finish(self, mu_, sigma_, mu, sigma):
        if np.abs(np.sum(mu_+sigma_) - np.sum(mu+sigma))<=1e-4:
            return True
        else:
            return False
        
        

In [119]:
gmm = GMM(k=2, maxIter=100)
mu, sigma, alpha = gmm.fit(Y)
print(mu)
print(sigma)
print(alpha)

[-2.03905042  0.48481547]
[0.45466645 1.03827373]
[0.29042801 0.70957199]
